In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import os


In [64]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [65]:

# Define transformations for the dataset
data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}


In [66]:

# Load datasets
data_dir = r'C:\Users\kara\Desktop\Project\New Plant Diseases Dataset(Augmented)'
datasets = {
    "train": datasets.ImageFolder(os.path.join(data_dir, "train"), data_transforms["train"]),
    "val": datasets.ImageFolder(os.path.join(data_dir, "valid"), data_transforms["val"]),
}

# Define dataloaders
dataloaders = {
    x: DataLoader(datasets[x], batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
    for x in ["train", "val"]
}

# Get class names
class_names = datasets["train"].classes
print(f"Classes: {class_names}")


Classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___

In [67]:

# Load a pre-trained model (ResNet18)
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(class_names))
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [68]:

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    print("-" * 10)

    for phase in ["train", "val"]:
        if phase == "train":
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:

            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == "train"):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == "train":
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(datasets[phase])
        epoch_acc = running_corrects.double() / len(datasets[phase])

        print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")


Epoch 1/10
----------
train Loss: 0.5081 Acc: 0.8748
val Loss: 0.0816 Acc: 0.9774
Epoch 2/10
----------
train Loss: 0.1537 Acc: 0.9554
val Loss: 0.0496 Acc: 0.9855
Epoch 3/10
----------
train Loss: 0.1148 Acc: 0.9650
val Loss: 0.0437 Acc: 0.9874
Epoch 4/10
----------
train Loss: 0.0940 Acc: 0.9712
val Loss: 0.0350 Acc: 0.9895
Epoch 5/10
----------
train Loss: 0.0819 Acc: 0.9747
val Loss: 0.0273 Acc: 0.9920
Epoch 6/10
----------
train Loss: 0.0740 Acc: 0.9772
val Loss: 0.0234 Acc: 0.9939
Epoch 7/10
----------
train Loss: 0.0698 Acc: 0.9790
val Loss: 0.0229 Acc: 0.9935
Epoch 8/10
----------
train Loss: 0.0622 Acc: 0.9807
val Loss: 0.0240 Acc: 0.9929
Epoch 9/10
----------
train Loss: 0.0575 Acc: 0.9821
val Loss: 0.0179 Acc: 0.9956
Epoch 10/10
----------
train Loss: 0.0545 Acc: 0.9831
val Loss: 0.0188 Acc: 0.9945


In [69]:

# Save the model
torch.save(model.state_dict(), "disease_model.pth")
print("Model saved as 'disease_model.pth'")

Model saved as 'plant_disease_model.pth'
